# Vitessce Widget Tutorial

# Visualization of a SpatialData object, blobs example

## Import dependencies


In [1]:
from spatialdata import read_zarr
from os.path import join

/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [2]:
spatialdata_filepath = join("..", "..", "..", "tissue-map-tools", "examples", "moffitt_multiscale.sdata.zarr")

In [3]:
sdata = read_zarr(spatialdata_filepath)

version mismatch: detected: RasterFormatV02, requested: FormatV04
/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [4]:
sdata

SpatialData object, with associated Zarr store: /Users/mkeller/research/dbmi/vitessce/tissue-map-tools/examples/moffitt_multiscale.sdata.zarr
├── Images
│     └── 'stains': DataTree[czyx] (2, 9, 9392, 5721), (2, 4, 4696, 2860), (2, 2, 2348, 1430)
├── Labels
│     ├── 'dapi_labels': DataTree[zyx] (9, 9392, 5721), (4, 4696, 2860), (2, 2348, 1430)
│     └── 'membrane_labels': DataTree[zyx] (9, 9392, 5721), (4, 4696, 2860), (2, 2348, 1430)
├── Points
│     ├── 'cells_centroids_cellpose': DataFrame with shape: (<Delayed>, 3) (3D points)
│     ├── 'molecule_baysor': DataFrame with shape: (<Delayed>, 21) (3D points)
│     └── 'molecules': DataFrame with shape: (<Delayed>, 12) (3D points)
├── Shapes
│     ├── 'cells_baysor': GeoDataFrame shape: (33222, 6) (2D shapes)
│     ├── 'cells_circles_baysor': GeoDataFrame shape: (5749, 2) (2D shapes)
│     ├── 'cells_layer_1_baysor': GeoDataFrame shape: (3797, 3) (2D shapes)
│     ├── 'cells_layer_2_baysor': GeoDataFrame shape: (4037, 3) (2D shapes)
│ 

In [5]:
sdata.points['molecules'].head()

,x,y,z,molecule_id,z_um,area,gene,brightness,x_um,qc_score,y_um,total_magnitude
0,1705.0,1271.0,0.0,1,2.5,4,Maoa,2.021306,-2935.386,0.954363,-1218.580,420.1126
1,1725.0,1922.0,0.0,2,2.5,4,Maoa,1.828640,-2933.229,0.908246,-1147.614,269.5874
2,1753.0,1863.0,0.0,3,2.5,5,Maoa,2.001268,-2930.104,0.977219,-1154.062,501.4615
3,1760.0,1865.0,0.0,4,2.5,7,Maoa,1.960428,-2929.339,0.991316,-1153.784,639.0364
4,1904.0,794.0,0.0,5,2.5,6,Maoa,1.937280,-2913.718,0.983210,-1270.474,519.3154


In [9]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    CoordinationLevel as CL,
    SpatialDataWrapper,
    get_initial_coordination_scope_prefix
)

## Configure Vitessce

Vitessce needs to know which pieces of data we are interested in visualizing, the visualization types we would like to use, and how we want to coordinate (or link) the views.

In [27]:
vc = VitessceConfig(
    schema_version="1.0.17",
    name='SpatialData with MERFISH data',
)
# Add data to the configuration:

dataset = vc.add_dataset(name='MERFISH').add_object(SpatialDataWrapper(
    sdata_store=spatialdata_filepath,
    # The following paths are relative to the root of the SpatialData zarr store on-disk.
    image_path="images/stains",
    coordinate_system="global",
    coordination_values={
        "fileUid": "stains"
    }
)).add_object(SpatialDataWrapper(
    sdata_store=spatialdata_filepath,
    # The following paths are relative to the root of the SpatialData zarr store on-disk.
    labels_path="labels/dapi_labels",
    coordinate_system="global",
    coordination_values={
        "obsType": "nucleus",
        "fileUid": "dapi"
    }
)).add_object(SpatialDataWrapper(
    sdata_store=spatialdata_filepath,
    # The following paths are relative to the root of the SpatialData zarr store on-disk.
    labels_path="labels/membrane_labels",
    coordinate_system="global",
    coordination_values={
        "obsType": "cell",
        "fileUid": "membrane"
    }
))

# Add views (visualizations) to the configuration:
spatial = vc.add_view("spatialBeta", dataset=dataset)
layer_controller = vc.add_view("layerControllerBeta", dataset=dataset)

vc.link_views_by_dict([spatial, layer_controller], {
    'imageLayer': CL([{
        "fileUid": "stains",
        'photometricInterpretation': 'BlackIsZero',
        'spatialLayerOpacity': 1.0,
        'spatialLayerVisible': True,
        'imageChannel': CL([
            {
                'spatialChannelVisible': True,
                "spatialTargetC": 0, # DAPI, Nucleus
                "spatialChannelColor": [0, 0, 255],
                "spatialChannelOpacity": 1.0
            },
            {
                'spatialChannelVisible': True,
                "spatialTargetC": 1, # Membrane, Cell
                "spatialChannelColor": [255, 255, 255],
                "spatialChannelOpacity": 1.0
            }
        ])
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "image"))

vc.link_views_by_dict([spatial, layer_controller], {
    'segmentationLayer': CL([{
        "fileUid": "membrane",
        'spatialLayerOpacity': 1.0,
        'spatialLayerVisible': True,
        'segmentationChannel': CL([{
            'spatialChannelVisible': True,
            'obsType': 'cell',
            "spatialChannelColor": [200, 200, 200],
            "obsColorEncoding": "spatialChannelColor",
        }]),
    }, {
        "fileUid": "dapi",
        'spatialLayerOpacity': 1.0,
        'spatialLayerVisible': True,
        'segmentationChannel': CL([{
            'spatialChannelVisible': True,
            'obsType': 'nucleus',
            "spatialChannelColor": [255, 255, 255],
            "obsColorEncoding": "spatialChannelColor",
        }]),
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "obsSegmentations"))

# Layout the views
vc.layout(spatial | layer_controller);

### Render the widget

In [28]:
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.17', 'name': 'SpatialData with MERFISH data', 'description': '', 'datas…